In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-10-23 21:30:19.842344: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-23 21:30:19.885763: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 21:30:19.885795: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 21:30:19.885820: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 21:30:19.893847: I tensorflow/core/platform/cpu_feature_g

## Load the Data for training

In [ ]:
signals=np.load('zenodo_data.npy')
seizure=np.load('zenodo_labels.npy')

X_train=signals
Y_train=np.reshape(seizure,(-1,1))

#shuffle X_train and Y_traint
indices=np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train=X_train[indices]
X_train=X_train[...,np.newaxis]
Y_train=Y_train[indices]
print(X_train.shape)
print(Y_train.shape)   

## Develope a large model

In [2]:
def block_type1(n_filters1,n_filters2, kernel, pad, x):
    block1=layers.Conv2D(n_filters1,kernel,padding=pad,activation='linear')(x)
    block1=layers.BatchNormalization()(block1)  
    block1=layers.Conv2D(n_filters1,kernel,padding=pad,activation='relu')(block1)
    block1=layers.Conv2D(n_filters1,kernel,padding=pad,activation='linear')(block1)
    block1=layers.BatchNormalization()(block1)
    block1=layers.Conv2D(n_filters1,kernel,padding=pad,activation='relu')(block1)
    block1=layers.Conv2D(n_filters2,kernel,padding=pad,activation='linear')(block1)
    block1=layers.BatchNormalization()(block1)

    block1_o=layers.Conv2D(n_filters2,kernel,padding=pad,activation='linear')(x)
    block1_o=layers.BatchNormalization()(block1_o)

    added1=layers.add([block1,block1_o])
    return layers.Conv2D(n_filters2,kernel,padding=pad,activation='relu')(added1)

def block_type2(n_filters1,n_filters2, kernel, pad, x):
    block=layers.Conv2D(n_filters1,kernel,padding=pad,activation='linear')(x)
    block=layers.BatchNormalization()(block)
    block=layers.Conv2D(n_filters1,kernel,padding=pad,activation='relu')(block)
    block=layers.Conv2D(n_filters1,kernel,padding=pad,activation='linear')(block)
    block=layers.BatchNormalization()(block)
    block=layers.Conv2D(n_filters1,kernel,padding=pad,activation='relu')(block)
    block=layers.Conv2D(n_filters2,kernel,padding=pad,activation='linear')(block)
    block=layers.BatchNormalization()(block)

    added=layers.add([x,block])
    return layers.Conv2D(n_filters2,kernel,padding=pad,activation='relu')(added)

In [3]:
Input=keras.Input(shape=(12,1024,1),name="Input_signal")
x=layers.Conv2D(32,(1,3),padding="same",activation='relu')(Input)
#batch normalization the previous layer
x=layers.BatchNormalization()(x)
x=layers.Conv2D(32,(1,3),padding="same",activation='relu')(x)
x=layers.MaxPool2D((1,2))(x)

x1=block_type1(32,128,(1,3),"same",x)
x2=block_type2(32,128,(1,3),"same",x1)
x3=block_type2(32,128,(1,3),"same",x2)
x3=layers.AveragePooling2D((1,4))(x3)  
x4=block_type1(64,256,(1,7),"same",x3)
x5=block_type2(64,256,(1,7),"same",x4)
x6=block_type2(64,256,(1,7),"same",x5)
x7=block_type2(64,256,(1,7),"same",x6)
x7=layers.AveragePooling2D((1,4))(x7)
x8=block_type1(128,512,(1,9),"same",x7)
x9=block_type2(128,512,(1,9),"same",x8)
x10=block_type2(128,512,(1,9),"same",x9)
x11=block_type2(128,512,(1,9),"same",x10)
x12=block_type2(128,512,(1,9),"same",x11)
x13=block_type2(128,512,(1,9),"same",x12)
x13=layers.AveragePooling2D((2,4))(x13)
x14=block_type1(256,512,(1,11),"same",x13)
x15=block_type2(256,512,(1,11),"same",x14)
x16=block_type2(256,512,(1,11),"same",x15)
x16=layers.GlobalAveragePooling2D()(x16)
x16=layers.Dense(4,activation='relu')(x16)
x16=layers.Dense(1,activation='sigmoid')(x16)




model=keras.Model(Input,x16)
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=['accuracy'])
model.summary()

2023-10-19 15:18:35.886020: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14774 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-10-19 15:18:35.886679: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14773 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:73:00.0, compute capability: 7.5


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input_signal (InputLayer)   [(None, 12, 1024, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 12, 1024, 32)         128       ['Input_signal[0][0]']        
                                                                                                  
 batch_normalization (Batch  (None, 12, 1024, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 conv2d_1 (Conv2D)           (None, 12, 1024, 32)         3104      ['batch_normalization[0][0

In [4]:
keras.utils.plot_model(model, "my_first_model.png", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Small model

In [ ]:
model.fit(X_train,Y_train,epochs=10,batch_size=16)

In [ ]:

# import tensorflow_addons as tfa
# from sklearn.model_selection import train_test_split

model=keras.Sequential(name="BrainNeoCare_CNN")
model.add(keras.layers.Conv2D(filters=32, kernel_size=(1,3), activation='relu', input_shape=(12,1024,1),name="Input_conv2D"))
model.add(keras.layers.MaxPooling2D((1,2),name="Maxpooling2D_1"))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(1,3), activation='relu',name="Conv2D_2"))
model.add(keras.layers.MaxPooling2D((1,2),name="Maxpooling2D_2"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(1,3), activation='relu',name="Conv2D_3"))
model.add(keras.layers.MaxPooling2D((1,2),name="Maxpooling2D_3"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(1,3), activation='relu',name="Conv2D_4"))
model.add(keras.layers.AveragePooling2D((1,2),name="AveragePooling2D_1"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu',name="2D_F_Con"))
model.add(keras.layers.AveragePooling2D((1,2),name="AveragePooling2D_2"))
model.add(keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu',name="2D_F_Con_2"))
model.add(keras.layers.MaxPooling2D((1,2),name="AveragePooling2D_3"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))  
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(8, activation='relu') )  
model.add(keras.layers.Dense(1, activation='sigmoid'))  

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, name="binary_crossentropy",)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  
model.summary()

In [ ]:
model.fit(X_train,Y_train,epochs=10,batch_size=32,validation_split=0.2,verbose=1)